In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

Read relevant data

In [ ]:
cpc = pd.read_csv("../midsave/cpc_com.csv", dtype={'code_com': str})

In [ ]:
cpc.shape

In [ ]:
map_df = gpd.read_file("../midsave/map_crime.gpkg")

In [ ]:
map_df.shape

In [ ]:
overture_gdf = gpd.read_file("../midsave/overture.gpkg").to_crs(map_df.crs)

In [ ]:
overture_gdf.shape

Define categories of interest

In [ ]:
categories_of_interest = ['school', 'church_cathedral', 'mosque', 'adult_entertainment', 
                          'amateur_sports_team', 'religious_organization']

Assign POIs to communes and aggregate number of categories per 1000 inhabitants

In [ ]:
poi = (gpd.sjoin(overture_gdf, map_df[['code_com', 'geometry']], predicate='within', how='left')
        .drop(columns = ['index_right'])
        .dropna(subset = ['code_com'])
        .query('category in @categories_of_interest')
        .pivot_table(index='code_com', columns='category', aggfunc='size', fill_value=0)
        .reset_index()
        .merge(cpc[['code_com', 'pop']], on = 'code_com', how = 'right')
        .fillna(0))

In [ ]:
for col in categories_of_interest:
    poi[col] = poi[col] / poi['pop'] * 1000

In [ ]:
poi.head(1)

In [ ]:
poi.drop(columns = ['pop'], inplace = True)

In [ ]:
poi.to_csv("../midsave/poi.csv", index=False)